# Zadanie: Analiza i prognozowanie ceny kakao z użyciem LSTM

## Cel zadania
Zbudowanie modelu sieci neuronowej LSTM (Long Short-Term Memory) do przewidywania cen kakao na podstawie danych historycznych.

## Dane wejściowe
Plik: `Daily Prices_Home_NEW.csv` zawierający:
- Date - data
- London futures (£ sterling/tonne) - ceny kontraktów futures w Londynie
- New York futures (US$/tonne) - ceny kontraktów futures w Nowym Jorku
- ICCO daily price (US$/tonne) - dzienna cena ICCO w USD (użyjemy tej kolumny)
- ICCO daily price (Euro/tonne) - dzienna cena ICCO w Euro

---
## Krok 1: Import bibliotek

**Zadanie:** Zaimportuj niezbędne biblioteki:
- `pandas` - do pracy z danymi tabelarycznymi
- `numpy` - do operacji na tablicach numerycznych
- `matplotlib.pyplot` - do wizualizacji
- `MinMaxScaler` z `sklearn.preprocessing` - do normalizacji danych
- `mean_squared_error, mean_absolute_error` z `sklearn.metrics` - do oceny modelu
- `Sequential` z `tensorflow.keras.models` - do budowy modelu sekwencyjnego
- `LSTM, Dense, Dropout` z `tensorflow.keras.layers` - warstwy sieci neuronowej
- `warnings` - do wyłączenia ostrzeżeń

In [ ]:
# TODO: Zaimportuj wszystkie potrzebne biblioteki



---
## Krok 2: Wczytanie i przygotowanie danych

**Zadanie:** 
1. Wczytaj dane z pliku CSV używając `pd.read_csv()`
2. Przekonwertuj kolumnę 'Date' na format datetime używając `pd.to_datetime()` z parametrem `format='%d/%m/%Y'`
3. Posortuj DataFrame po dacie (od najstarszych) używając `.sort_values()` i zresetuj indeks
4. Wyczyść dane w kolumnie 'ICCO daily price (US$/tonne)':
   - Usuń przecinki używając `.str.replace(',', '')`
   - Przekonwertuj na typ float używając `.astype(float)`
5. Wyświetl zakres dat, liczbę rekordów i pierwsze wiersze

In [ ]:
# TODO: Wczytaj dane
df = 

# TODO: Konwertuj kolumnę 'Date' na datetime


# TODO: Posortuj DataFrame po dacie (od najstarszych)


# TODO: Wyczyść dane - usuń przecinki i przekonwertuj na float


# TODO: Wyświetl podstawowe informacje
print(f"Zakres dat: {df['Date'].min()} - {df['Date'].max()}")
print(f"Liczba rekordów: {len(df)}")
print(f"\nPierwsze wiersze:")


---
## Krok 3: Wizualizacja danych historycznych

**Zadanie:**
1. Stwórz wykres liniowy ceny kakao w czasie
2. Ustaw rozmiar wykresu na (14, 6) używając `plt.figure(figsize=(14, 6))`
3. Dodaj tytuł, etykiety osi, siatkę
4. Obróć etykiety osi X o 45 stopni dla lepszej czytelności
5. Wyświetl statystyki opisowe używając `.describe()`

In [ ]:
# TODO: Stwórz wykres ceny kakao w czasie









# TODO: Wyświetl statystyki opisowe
print(f"\nStatystyki opisowe:")


---
## Krok 4: Przygotowanie danych dla LSTM

**Zadanie:**
1. Wybierz kolumnę z cenami i przekształć ją do kształtu (-1, 1) używając `.reshape(-1, 1)`
2. Znormalizuj dane do zakresu [0, 1] używając `MinMaxScaler`:
   - **Dlaczego normalizujemy?** Sieci neuronowe uczą się lepiej, gdy dane są w podobnej skali
   - **feature_range=(0, 1)** - wszystkie wartości będą w przedziale [0, 1]
3. Podziel dane na zbiór treningowy i testowy w proporcji 80/20:
   - **80% treningowy** - model uczy się na tych danych
   - **20% testowy** - sprawdzamy jak model radzi sobie z nieznanymi danymi

In [ ]:
# TODO: Wybierz kolumnę z cenami i przekształć do kształtu (-1, 1)
data = 

# TODO: Stwórz obiekt MinMaxScaler i znormalizuj dane
scaler = 
data_scaled = 

# TODO: Podziel dane na zbiór treningowy (80%) i testowy (20%)
train_size = 
train_data = 
test_data = 

print(f"Rozmiar zbioru treningowego: {len(train_data)}")
print(f"Rozmiar zbioru testowego: {len(test_data)}")

---
## Krok 5: Tworzenie sekwencji dla LSTM

**Zadanie:**
1. Zdefiniuj funkcję `create_sequences(data, seq_length)`, która:
   - Tworzy sekwencje danych o długości `seq_length`
   - Zwraca X (dane wejściowe) i y (wartości docelowe)
   - **Przykład:** dla seq_length=30, model używa 30 dni wstecz do przewidzenia dnia następnego

2. Ustaw **SEQ_LENGTH = 30** (liczba dni wykorzystywanych do predykcji)
   - **Dlaczego 30?** To kompromis między:
     - Za mało: model ma za mało kontekstu
     - Za dużo: model może się przeuczyć i będzie wolniejszy

3. Stwórz sekwencje dla danych treningowych i testowych

In [ ]:
# TODO: Zdefiniuj funkcję do tworzenia sekwencji
def create_sequences(data, seq_length):
    X, y = [], []
    # TODO: Uzupełnij pętlę, która tworzy sekwencje
    for i in range(len(data) - seq_length):
        # Dodaj sekwencję do X
        
        # Dodaj wartość docelową do y
        
    return np.array(X), np.array(y)

# TODO: Ustaw długość sekwencji
SEQ_LENGTH = 

# TODO: Stwórz sekwencje dla zbioru treningowego i testowego
X_train, y_train = 
X_test, y_test = 

print(f"Kształt X_train: {X_train.shape}")
print(f"Kształt y_train: {y_train.shape}")
print(f"Kształt X_test: {X_test.shape}")
print(f"Kształt y_test: {y_test.shape}")

---
## Krok 6: Budowa modelu LSTM

**Zadanie:** Zbuduj model Sequential z następującymi warstwami:

### Parametry warstw:

1. **LSTM(64, return_sequences=True, input_shape=(SEQ_LENGTH, 1))**
   - **64** - liczba jednostek LSTM (neuronów) w warstwie
   - **return_sequences=True** - zwraca pełną sekwencję (potrzebne dla kolejnej warstwy LSTM)
   - **input_shape=(SEQ_LENGTH, 1)** - kształt danych wejściowych (30 kroków czasowych, 1 cecha)

2. **Dropout(0.2)**
   - **0.2** - wyłącza losowo 20% neuronów podczas treningu
   - **Cel:** zapobieganie przeuczeniu (overfitting)

3. **LSTM(32, return_sequences=False)**
   - **32** - mniej neuronów niż w pierwszej warstwie
   - **return_sequences=False** - zwraca tylko ostatni wynik (nie potrzebujemy sekwencji)

4. **Dropout(0.2)**

5. **Dense(16, activation='relu')**
   - **16** - liczba neuronów w warstwie gęstej
   - **activation='relu'** - funkcja aktywacji ReLU (Rectified Linear Unit)

6. **Dense(1)**
   - **1** - warstwa wyjściowa z jednym neuronem (przewidywana cena)

### Kompilacja modelu:
- **optimizer='adam'** - algorytm optymalizacji Adam (adaptacyjne tempo uczenia)
- **loss='mse'** - funkcja straty Mean Squared Error
- **metrics=['mae']** - dodatkowa metryka Mean Absolute Error

In [ ]:
# TODO: Zbuduj model LSTM
model = Sequential([
    # TODO: Dodaj pierwszą warstwę LSTM z 64 neuronami
    
    # TODO: Dodaj Dropout(0.2)
    
    # TODO: Dodaj drugą warstwę LSTM z 32 neuronami
    
    # TODO: Dodaj Dropout(0.2)
    
    # TODO: Dodaj warstwę Dense z 16 neuronami i aktywacją 'relu'
    
    # TODO: Dodaj warstwę wyjściową Dense z 1 neuronem
    
])

# TODO: Skompiluj model z optimizer='adam', loss='mse', metrics=['mae']


# Wyświetl podsumowanie modelu
model.summary()

---
## Krok 7: Trenowanie modelu

**Zadanie:** Wytrenuj model używając metody `.fit()`

### Parametry treningu:
- **epochs=50** - liczba pełnych przejść przez zbiór treningowy
  - Więcej epok = dłuższe uczenie, ale lepsze dopasowanie
  - Za dużo epok = ryzyko przeuczenia
  
- **batch_size=16** - liczba próbek przetwarzanych przed aktualizacją wag
  - Mniejszy batch = więcej aktualizacji, ale bardziej "hałaśliwe" uczenie
  - Większy batch = stabilniejsze uczenie, ale może utknąć w minimum lokalnym
  
- **validation_split=0.1** - 10% danych treningowych użytych do walidacji
  - Pozwala monitorować czy model się nie przeuczy
  
- **verbose=1** - wyświetla pasek postępu podczas treningu

In [ ]:
# TODO: Wytrenuj model
history = model.fit(
    # TODO: Dodaj dane treningowe X_train, y_train
    
    # TODO: Ustaw liczbę epok na 50
    
    # TODO: Ustaw batch_size na 16
    
    # TODO: Ustaw validation_split na 0.1
    
    # TODO: Ustaw verbose na 1
    
)

---
## Krok 8: Wizualizacja procesu uczenia

**Zadanie:** Stwórz dwa wykresy pokazujące:
1. Funkcję straty (Loss) dla zbioru treningowego i walidacyjnego
2. Metrykę MAE dla zbioru treningowego i walidacyjnego

**Wskazówka:** Użyj `history.history['loss']`, `history.history['val_loss']`, etc.

In [ ]:
# TODO: Stwórz figurę z dwoma wykresami obok siebie
plt.figure(figsize=(14, 5))

# TODO: Pierwszy wykres - funkcja straty
plt.subplot(1, 2, 1)





# TODO: Drugi wykres - MAE
plt.subplot(1, 2, 2)





plt.tight_layout()
plt.show()

---
## Krok 9: Predykcje i ewaluacja modelu

**Zadanie:**
1. Wykonaj predykcje na zbiorze testowym używając `model.predict()`
2. Odwróć normalizację predykcji i prawdziwych wartości używając `scaler.inverse_transform()`
3. Oblicz metryki:
   - **MSE** (Mean Squared Error) - średni kwadrat błędu
   - **RMSE** (Root MSE) - pierwiastek z MSE, w tych samych jednostkach co cena
   - **MAE** (Mean Absolute Error) - średni błąd bezwzględny
   - **MAPE** (Mean Absolute Percentage Error) - średni procentowy błąd bezwzględny

In [ ]:
# TODO: Wykonaj predykcje na zbiorze testowym
predictions = 

# TODO: Odwróć normalizację
predictions_rescaled = 
y_test_rescaled = 

# TODO: Oblicz metryki
mse = 
rmse = 
mae = 
mape = 

print(f"Metryki modelu:")
print(f"MSE: {mse:.2f}")
print(f"RMSE: {rmse:.2f} US$/tonne")
print(f"MAE: {mae:.2f} US$/tonne")
print(f"MAPE: {mape:.2f}%")

---
## Krok 10: Wizualizacja predykcji vs rzeczywiste wartości

**Zadanie:** Stwórz wykres liniowy porównujący:
- Rzeczywiste ceny (z y_test_rescaled)
- Predykcje modelu (z predictions_rescaled)

In [ ]:
# TODO: Stwórz wykres porównujący predykcje z rzeczywistymi wartościami









---
## Krok 11: Wykres rozrzutu (scatter plot)

**Zadanie:** Stwórz wykres rozrzutu pokazujący:
- Oś X: rzeczywiste wartości
- Oś Y: przewidywane wartości
- Czerwona linia: idealna predykcja (y=x)

**Interpretacja:** Im bliżej punkty są do czerwonej linii, tym lepszy model

In [ ]:
# TODO: Stwórz wykres rozrzutu









---
## Krok 12: Predykcja przyszłych wartości

**Zadanie:**
1. Zdefiniuj funkcję `predict_next_days()`, która:
   - Bierze model, ostatnią sekwencję, liczbę dni do przewidzenia i scaler
   - W pętli:
     - Przewiduje następny dzień
     - Aktualizuje sekwencję (usuwa pierwszy element, dodaje przewidywaną wartość)
   - Zwraca przewidziane ceny po odwróceniu normalizacji

2. Przewidź ceny na 14 dni w przód

In [ ]:
# TODO: Zdefiniuj funkcję do przewidywania przyszłych dni
def predict_next_days(model, last_sequence, n_days, scaler):
    predictions = []
    current_sequence = last_sequence.copy()
    
    # TODO: Uzupełnij pętlę przewidującą kolejne dni
    for _ in range(n_days):
        # Przewidź następny dzień
        
        
        # Zaktualizuj sekwencję
        
    
    # TODO: Odwróć normalizację
    predictions = 
    return predictions

# TODO: Pobierz ostatnią sekwencję z danych
last_sequence = 

# TODO: Przewiduj na 14 dni w przód
future_days = 14
future_predictions = 

print(f"\nPredykcje na kolejne {future_days} dni:")
for i, pred in enumerate(future_predictions, 1):
    print(f"Dzień +{i}: {pred[0]:.2f} US$/tonne")

---
## Krok 13: Wizualizacja predykcji przyszłych wartości

**Zadanie:** Stwórz wykres pokazujący:
- Niebieską linię: ostatnie 30 dni danych historycznych
- Czerwoną linię przerywaną: 14 dni predykcji
- Zieloną linię pionową: dzień dzisiejszy (granica między danymi a predykcją)

In [ ]:
# TODO: Pobierz ostatnie 30 dni
last_30_days = 
all_values = np.concatenate([last_30_days, future_predictions])

# TODO: Stwórz wykres z danymi historycznymi i predykcjami
plt.figure(figsize=(14, 6))

# Dane historyczne


# Predykcje


# Linia "dziś"





plt.tight_layout()
plt.show()

---
## Zadania dodatkowe (dla ambitnych!)

1. **Eksperymentuj z parametrami:**
   - Zmień SEQ_LENGTH (np. 10, 60, 90) i zobacz jak wpływa to na wyniki
   - Zmień liczbę neuronów w warstwach LSTM (np. 128, 64, 32)
   - Zmień liczbę epok (np. 30, 100, 200)
   - Zmień batch_size (np. 8, 32, 64)

2. **Dodaj więcej warstw:**
   - Spróbuj dodać trzecią warstwę LSTM
   - Dodaj więcej warstw Dense

3. **Użyj innych kolumn:**
   - Spróbuj użyć ceny z Londynu lub Nowego Jorku
   - Użyj wielu kolumn jednocześnie (multi-feature LSTM)

4. **Zapisz i wczytaj model:**
   - Użyj `model.save('cocoa_model.h5')` do zapisania
   - Użyj `load_model('cocoa_model.h5')` do wczytania

5. **Porównaj z innymi metodami:**
   - Zbuduj prosty model używając GRU zamiast LSTM
   - Spróbuj użyć prostszego modelu (np. Linear Regression) jako baseline

---

## Podsumowanie

Gratulacje! Po wykonaniu wszystkich kroków powinieneś mieć:
- Działający model LSTM do przewidywania cen kakao
- Wizualizacje pokazujące jak dobrze model działa
- Predykcje na przyszłość
- Zrozumienie parametrów i architektury sieci LSTM

### Kluczowe wnioski:
- LSTM dobrze radzi sobie z danymi sekwencyjnymi (szeregi czasowe)
- Normalizacja danych jest kluczowa dla wydajności sieci neuronowych
- Dropout pomaga zapobiegać przeuczeniu
- Podział na zbiór treningowy/testowy/walidacyjny pozwala ocenić model
- Predykcja przyszłych wartości bazuje na ostatnich znanych danych